In [151]:
import pandas as pd

In [166]:
df=pd.read_csv('final_relationship_finale.csv')
df_non=pd.read_csv('final_nonML_finale.csv')

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])


In [198]:
fan_in_accounts_dict

{0: 'Bang Windah Bersaudara',
 1: 'Olivia Harris',
 2: 'Dian Arifin',
 3: 'Michael Walker',
 4: 'Yusuf Arifin'}

In [209]:
dict(enumerate(['a','b']),start=1)

{0: 'a', 1: 'b', 'start': 1}

In [226]:

# Function to detect fan-in and fan-out
def detect_fan_in_out(df, last_n_month=1, min_transactions=3):
    period=f'{last_n_month}M'
    fan_in_accounts = set()
    fan_out_accounts = set()
    
    for account in df['RECEIVER_ACCOUNT'].unique():
        account_df = df[df['RECEIVER_ACCOUNT'] == account]
        account_df = account_df.set_index('TIMESTAMP').resample(period).size()
        if any(account_df >= min_transactions):
            fan_in_accounts.add(account)
    fan_in_accounts_dict=dict(enumerate(fan_in_accounts,start=1)) #mvp solution using just the name, not the trx ids
    print(fan_in_accounts_dict)
    for account in df['SENDER_ACCOUNT'].unique():
        account_df = df[df['SENDER_ACCOUNT'] == account]
        account_df = account_df.set_index('TIMESTAMP').resample(period).size()
        if any(account_df >= min_transactions):
            fan_out_accounts.add(account)
    fan_out_accounts_dict=dict(enumerate(fan_out_accounts,start=1)) #mvp solution using just the name, not the trx ids

    # df['fan-in_flag'] = df['RECEIVER_ACCOUNT'].apply(lambda x: 1 if x in fan_in_accounts else 0)
    # df['fan-out_flag'] = df['SENDER_ACCOUNT'].apply(lambda x: 1 if x in fan_out_accounts else 0)
    
    reverse_fan_in_mapping = {v: k for k, v in fan_in_accounts_dict.items()}
    df['fan-in_flag'] = df['RECEIVER_ACCOUNT'].map(reverse_fan_in_mapping).fillna(0)

    # Create a reverse mapping dictionary
    reverse_fan_out_mapping = {v: k for k, v in fan_out_accounts_dict.items()}
    df['fan-out_flag'] = df['SENDER_ACCOUNT'].map(reverse_fan_out_mapping).fillna(0)

    # Fill NaN values with 0 for accounts not in fan-in or fan-out lists
    df['fan-in_flag'].fillna(0, inplace=True)
    df['fan-out_flag'].fillna(0, inplace=True)
    return df


def scatter_gather_flag(df,last_n_month=1):
    scatter_gather_flags = []
    for index, row in df.iterrows():
        if row['fan-out_flag'] !=0:
            # Check for fan-in within the next month for the sender of this transaction
            future_fan_in = df[(df['SENDER_ACCOUNT'] == row['RECEIVER_ACCOUNT']) & 
                            (df['TIMESTAMP'] > row['TIMESTAMP']) & 
                            (df['TIMESTAMP'] <= row['TIMESTAMP'] + pd.DateOffset(months=last_n_month)) & 
                            (df['fan-in_flag'] !=0)]
            
            if not future_fan_in.empty:
                scatter_gather_flags.append(row['fan-out_flag']) #scatter itu fan out dl
            else:
                scatter_gather_flags.append(0)
        else:
            scatter_gather_flags.append(0)
    df['scatter-gather_flag'] = scatter_gather_flags

    for index, row in df.iterrows():
        if row['fan-in_flag'] !=0 and row['scatter-gather_flag'] == 0:
            # Check for fan-out within the previous month for the receiver of this transaction
            past_fan_out = df[(df['RECEIVER_ACCOUNT'] == row['SENDER_ACCOUNT']) & 
                            (df['TIMESTAMP'] < row['TIMESTAMP']) & 
                            (df['TIMESTAMP'] >= row['TIMESTAMP'] - pd.DateOffset(months=last_n_month)) & 
                            (df['fan-out_flag'] !=0)]
            if not past_fan_out.empty:
                print('\n\n\naaa',past_fan_out['fan-out_flag'].iloc[0])
                
                df.at[index, 'scatter-gather_flag'] = past_fan_out['fan-out_flag'].iloc[0]
    return df

def gather_scatter_flag(df, last_n_month=2):
    gather_scatter_flags = []

    for index, row in df.iterrows():
        if row['fan-in_flag'] != 0:
            # Check for fan-out within the next month for the receiver of this transaction
            future_fan_out = df[(df['SENDER_ACCOUNT'] == row['RECEIVER_ACCOUNT']) & 
                                (df['TIMESTAMP'] > row['TIMESTAMP']) & 
                                (df['TIMESTAMP'] <= row['TIMESTAMP'] + pd.DateOffset(months=last_n_month)) & 
                                (df['fan-out_flag'] !=0)]
            
            if not future_fan_out.empty:
                gather_scatter_flags.append(row['fan-in_flag'])
            else:
                gather_scatter_flags.append(0)
        else:
            gather_scatter_flags.append(0)
    df['gather-scatter_flag'] = gather_scatter_flags

    for index, row in df.iterrows():
        if row['fan-out_flag'] !=0 and row['gather-scatter_flag'] == 0:
            # Check for fan-in within the previous month for the sender of this transaction
            past_fan_in = df[(df['RECEIVER_ACCOUNT'] == row['SENDER_ACCOUNT']) & 
                             (df['TIMESTAMP'] < row['TIMESTAMP']) & 
                             (df['TIMESTAMP'] >= row['TIMESTAMP'] - pd.DateOffset(months=last_n_month)) & 
                             (df['fan-in_flag'] != 0)]
            
            if not past_fan_in.empty:
                df.at[index, 'gather-scatter_flag'] = past_fan_in['fan-in_flag'].iloc[0]
    return df

# Detect fan-in and fan-out patterns
df = detect_fan_in_out(df)
df = scatter_gather_flag(df)
df = gather_scatter_flag(df)


C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557

C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557

{1: 'Bang Windah Bersaudara', 2: 'Olivia Harris', 3: 'Michael Walker', 4: 'Yusuf Arifin', 5: 'Dian Arifin'}


C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\2557768020.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  account_df = account_df.set_index('TIMESTAMP').resample(period).size()
C:\Users\BNI\AppData\Local\Temp\ipykernel_14756\




aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0



aaa 7.0


In [227]:
df[df['ML_ID']=='gather-scatter_2']


,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE,fan-in_flag,fan-out_flag,scatter-gather_flag,gather-scatter_flag
52,Yusuf Arifin,Zainal Abidin,IDR,4687.0,9.974400e+07,2022-08-01 13:07:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
53,Yusuf Arifin,Dewi Anggraini,IDR,2416.0,9.113013e+07,2022-08-02 13:15:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
54,Yusuf Arifin,Feri Widodo,IDR,16327.0,9.058118e+07,2022-08-02 13:43:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
55,Yusuf Arifin,Gilang Saputra,IDR,14.0,8.479448e+07,2022-08-06 21:39:00,gather-scatter_2,wire,4.0,0.0,7.0,4.0
56,Yusuf Arifin,Hana Rahayu,IDR,1512.0,9.025696e+07,2022-08-07 16:18:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
57,Yusuf Arifin,Indah Pratiwi,IDR,211621.0,9.689204e+07,2022-08-21 18:00:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
58,Yusuf Arifin,Joko Prasetyo,IDR,25.0,8.654277e+07,2022-08-23 16:32:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
59,Yusuf Arifin,Kiki Amalia,IDR,220.0,8.239526e+07,2022-08-26 00:20:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
60,Yusuf Arifin,Lina Suryani,IDR,4292.0,9.047420e+07,2022-08-27 11:26:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0
61,Yusuf Arifin,Mira Andriani,IDR,26967.0,8.157306e+07,2022-08-27 21:02:00,gather-scatter_2,wire,4.0,0.0,0.0,4.0


In [ ]:
fan_in_accounts_dict

{0: 'Bang Windah Bersaudara',
 1: 'Olivia Harris',
 2: 'Dian Arifin',
 3: 'Michael Walker',
 4: 'Yusuf Arifin'}

In [106]:
df[df['ML_ID'].str.startswith('s')].head(50)

,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE,gather-scatter flag,Fan_In_Flag,Fan_Out_Flag,Scatter_Gather_Flag
170,Bayu Kurnia,Chandra Surya,IDR,23435.0,8.722236e+07,2022-08-01 05:41:00,scatter-gather_1,wire,0,No,Yes,Yes
172,Elang Wijaya,Chandra Surya,USD,35862.0,5.115950e+04,2022-08-02 13:14:00,scatter-gather_1,wire,0,No,Yes,Yes
174,Feni Rahman,Chandra Surya,RM,17291.0,2.900000e+04,2022-08-03 12:42:00,scatter-gather_1,wire,0,No,Yes,Yes
176,Gilang Saputra,Chandra Surya,RM,2044.0,2.900000e+04,2022-08-05 18:24:00,scatter-gather_1,wire,0,No,Yes,Yes
178,Hendro Setiawan,Chandra Surya,RM,16902.0,2.900000e+04,2022-08-10 04:05:00,scatter-gather_1,wire,0,No,Yes,Yes
180,Ika Permata,Chandra Surya,RM,27656.0,2.900000e+04,2022-08-10 09:39:00,scatter-gather_1,wire,0,No,Yes,Yes
182,Jaya Mahendra,Chandra Surya,IDR,29556.0,8.701991e+07,2022-08-12 12:16:00,scatter-gather_1,wire,0,No,Yes,Yes
184,Krisna Wijaya,Chandra Surya,IDR,239258.0,9.382439e+07,2022-08-13 03:52:00,scatter-gather_1,wire,0,No,Yes,Yes
186,Lukman Hakim,Chandra Surya,IDR,135332.0,9.352420e+07,2022-08-17 06:07:00,scatter-gather_1,wire,0,No,Yes,Yes
188,Maya Sari,Chandra Surya,RM,22483.0,2.900000e+04,2022-08-18 05:50:00,scatter-gather_1,wire,0,No,Yes,Yes


In [84]:
df=df.sort_values(['ML_ID','TIMESTAMP'])

In [68]:
df.query("ML_ID.str.startswith('scatt')")#.drop(columns=['gather-scatter flag','Suspicious_Gather_Scatter','Suspicious_Scatter_Gather'])

,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE,gather-scatter flag,Fan_In_Flag,Fan_Out_Flag,Scatter_Gather_Flag
170,Bayu Kurnia,Chandra Surya,IDR,23435.0,8.722236e+07,2022-08-01 05:41:00,scatter-gather_1,wire,0,No,No,No
171,Dian Arifin,Bayu Kurnia,USD,117909.0,5.943043e+04,2022-08-19 18:03:00,scatter-gather_1,wire,0,No,No,No
172,Elang Wijaya,Chandra Surya,USD,35862.0,5.115950e+04,2022-08-02 13:14:00,scatter-gather_1,wire,0,No,No,No
173,Dian Arifin,Elang Wijaya,USD,117909.0,5.278837e+04,2022-08-25 06:41:00,scatter-gather_1,wire,0,No,No,No
174,Feni Rahman,Chandra Surya,RM,17291.0,2.900000e+04,2022-08-03 12:42:00,scatter-gather_1,wire,0,No,No,No
175,Dian Arifin,Feni Rahman,USD,117909.0,5.996315e+04,2022-08-28 09:27:00,scatter-gather_1,wire,0,No,No,No
176,Gilang Saputra,Chandra Surya,RM,2044.0,2.900000e+04,2022-08-05 18:24:00,scatter-gather_1,wire,0,No,Yes,No
177,Dian Arifin,Gilang Saputra,USD,117909.0,5.282886e+04,2022-08-28 14:20:00,scatter-gather_1,wire,0,Yes,No,No
178,Hendro Setiawan,Chandra Surya,RM,16902.0,2.900000e+04,2022-08-10 04:05:00,scatter-gather_1,wire,0,No,Yes,No
179,Dian Arifin,Hendro Setiawan,USD,117909.0,5.932239e+04,2022-08-29 08:48:00,scatter-gather_1,wire,0,Yes,No,No


In [128]:
df.query("gather-scatter_flag.str.startswith('Y')")#.drop(columns=['gather-scatter flag','Suspicious_Gather_Scatter','Suspicious_Scatter_Gather'])

UndefinedVariableError: name 'gather' is not defined

In [19]:
df[df['ML_ID'].str[:14]=='gather-scatter'].drop(columns=['ML_ID']).head(30)

,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,TX_TYPE
0,Bang Windah Bersaudara,Aliya Zahra,IDR,0.0,8.528834e+07,2022-08-01 11:40:00,wire
1,Bang Windah Bersaudara,Ratna Sari,IDR,3.0,9.974407e+07,2022-08-05 13:16:00,wire
2,Bang Windah Bersaudara,Dinda Puspita,IDR,718.0,8.912334e+07,2022-08-07 09:23:00,wire
3,Bang Windah Bersaudara,Sulastri,IDR,11.0,9.654125e+07,2022-08-08 11:24:00,wire
4,Bang Windah Bersaudara,Dewi Lestari,IDR,0.0,8.163624e+07,2022-08-16 14:39:00,wire
5,Bang Windah Bersaudara,Rina Kumala,IDR,3.0,8.833087e+07,2022-08-21 00:38:00,wire
6,Bang Windah Bersaudara,Bunga Cempaka,IDR,1.0,9.630454e+07,2022-08-22 05:10:00,wire
7,Bang Windah Bersaudara,Indah Permata,IDR,21418.0,9.723188e+07,2022-09-05 05:12:00,wire
8,Siti Nurhaliza,Bang Windah Bersaudara,EUR,1831.0,5.035374e+04,2022-09-05 00:41:00,wire
9,Mutiara Hati,Bang Windah Bersaudara,IDR,1831.0,8.230440e+07,2022-09-06 02:57:00,wire


In [7]:
df_non.columns=[c.upper() for c in df_non.columns]
df_non

,RECEIVER_ACCOUNT,SENDER_ACCOUNT,CURRENCY,TX_ID,TX_AMOUNT,TIMESTAMP,ML_ID,TX_TYPE
0,Edward Morrison,John Lowery,IDR,0,2549.61,2022-09-02 15:21:00,NaN,cheque
1,Danielle Mcbride,Angela Miller,INR,1,2159711.37,2022-09-01 18:16:00,NaN,cheque
2,Hunter Robinson,Michelle Holmes,BRL,2,246088.70,2022-09-02 14:22:00,NaN,cheque
3,Jessica Rodriguez,Joshua Best,AUD,3,124.25,2022-09-09 15:13:00,NaN,cheque
4,Jenna Ryan,Jenna Ryan,IDR,4,13.72,2022-09-01 00:12:00,NaN,reinvestment
...,...,...,...,...,...,...,...,...
995,Alicia Skinner,Alicia Skinner,IDR,995,74773.66,2022-09-02 16:28:00,NaN,wire
996,Dr. Kendra Mitchell,Ethan Tanner,IDR,996,131.28,2022-09-09 06:15:00,NaN,credit card
997,Linda Hall,Erin Aguilar,USD,997,224.83,2022-09-09 08:42:00,NaN,cheque
998,Daniel Jones,Brooke Ware,IDR,998,1914.48,2022-09-05 10:12:00,NaN,cheque
